1.먼저 F 빼고 다 지웁시다.

In [1]:
import json
import os
import math
import pandas as pd
import cv2

- data 디렉토리는 설정해주세요. 저같은 경우는 ../data/에 있습니다.

In [2]:
file_dir = "../data/" # setting

- dir 세팅

In [3]:
step1 = os.listdir(file_dir)
step1

['CROWD_REAL005', 'img', 'CROWD_REAL006', 'CROWD_REAL004']

- 차례차례 들어가볼게요

In [ ]:
for dir1 in step1:
    step1_dir = file_dir + dir1
    step2 = os.listdir(step1_dir)
#     print(step1_dir)
    
    for dir2 in step2:
        step2_dir = step1_dir + '/' + dir2
        step3 = os.listdir(step2_dir)
        
        for dir3 in step3:
            step3_dir = step2_dir + '/' + dir3
            file_list = os.listdir(step3_dir)
            
            ## 여기 file list 파싱해서 F 다 지웁시다.
            for file in file_list:
                file_name = step3_dir + '/' + file
                if file.split('.')[0].split('_')[4] != 'F': # 삭제
                    os.remove(file_name)
#                     print(file_name) # 지워진 파일 출력

2. 영상 잘라서 이미지 만들고 키포인트Json 삭제

- 데이터가 비어있는 영상도 체크합시다. 
- 저희는 [영상제목, 영상 시작시간, 영상 종료시간] 정보로 이미지 만들게요.

In [76]:
video_start_end_info = []
json_start_end_info = []
error_video = []

In [77]:
for dir1 in step1:
    step1_dir = file_dir + dir1
    step2 = os.listdir(step1_dir)
#     print(step1_dir)
    
    for dir2 in step2:
        step2_dir = step1_dir + '/' + dir2
        step3 = os.listdir(step2_dir)
        
        # 형태소로 들어가기 및 영상 저장
        video_dir = step2_dir + '/' + step3[0]
        keypoints_dir = step2_dir + '/' + step3[1]
        step3_dir = step2_dir + '/' + step3[2]
        file_list = os.listdir(step3_dir)
        
        print("파일 갯수", len(file_list))
        
        for file in file_list:
            tmp = []
            tmp2 = []
            
            file_name = step3_dir + '/' + file
#             print(file_name)
            
            with open(file_name) as json_file:
                try:
                    json_data = json.load(json_file)
                    clip_name = json_data["metaData"]['name']
    #                 print(clip_name)
    
                    if len(json_data["data"]) != 0:
                        clip_start = json_data["data"][0]['start']
                        tmp.append(video_dir + '/' + clip_name)
                        tmp2.append(keypoints_dir + '/' + clip_name)
                    else:
                        print("에러난 파일명: ", clip_name)
                        error_video.append(file_name)
                        continue
                    
                    clip_start = math.trunc(clip_start * 10) / 10
                    tmp.append(clip_start)
                    tmp2.append(clip_start)
                #     print(clip_start)

                    clip_end = json_data["data"][0]['end']
                    clip_end = math.ceil(clip_end * 10) / 10
                    tmp.append(clip_end)
                    tmp2.append(clip_end)
                #     print(clip_end)
                except:
                    print("에러난 파일명: ", clip_name) # 아마 data가 없는파일임.
            video_start_end_info.append(tmp)
            json_start_end_info.append(tmp2)

파일 갯수 999
파일 갯수 1000
파일 갯수 1000
파일 갯수 1000
파일 갯수 1000
파일 갯수 999
파일 갯수 1000
파일 갯수 1000
파일 갯수 1000


- 갯수 체크하시구

In [78]:
print("정상적으로 담긴거 ", len(video_start_end_info))
print("에러 영상 ", len(error_video))

정상적으로 담긴거  8998
에러 영상  0


- 에러난 영상 지우기, 전 이미 지워버렸지만 확인해주세요

In [79]:
for err in error_video:
    os.remove(err)

- 잘 담겼는지 확인해주세요~

In [80]:
pd_test = pd.DataFrame(video_start_end_info)

pd_test.head()

,0,1,2
0,../data/CROWD_REAL005/CROWD_REAL015/영상/NIA_SL_...,1.5,6.4
1,../data/CROWD_REAL005/CROWD_REAL015/영상/NIA_SL_...,1.0,7.8
2,../data/CROWD_REAL005/CROWD_REAL015/영상/NIA_SL_...,1.5,5.4
3,../data/CROWD_REAL005/CROWD_REAL015/영상/NIA_SL_...,1.9,6.7
4,../data/CROWD_REAL005/CROWD_REAL015/영상/NIA_SL_...,1.1,5.4


- 파일이 뭐같아서 MP4랑 mp4 차이가 있으면 인식을 못하더라구요. 그래서 올 소문자로 확장자를 바꿔야합니다 ㅠ

In [81]:
for video_info in video_start_end_info:
    if video_info[0].split('.')[-1] != "mp4":
        tmp = video_info[0].split('.')
        tmp[-1] = "mp4"
        edit_video_dir = '.'.join(tmp)
        video_info[0] = edit_video_dir
#         print(edit_video_dir)


for video_info in json_start_end_info:
    if video_info[0].split('.')[-1] != "mp4":
        tmp = video_info[0].split('.')
        tmp[-1] = "mp4"
        edit_video_dir = '.'.join(tmp)
        video_info[0] = edit_video_dir
#         print(edit_video_dir)

- 이미지를 저장할 폴더를 만들고 잘라봅시다.

In [85]:
img_dir = "../data/img"
os.makedirs(img_dir)

- 시작합니다~ 추가로 한번 진행해보다가 중단하시고 프레임번호가 위에 영상 시작, 끝시간과 맞는지 확인해보는거도 조아요~

In [ ]:
for video_info in video_start_end_info:
    # video_info[0] : 영상 경로, video_info[1] : 영상 시작, video_info[2] : 영상 끝
    
    editing_video = video_info[0].split('/')[-1]
    output_dir = img_dir + '/' + editing_video
    os.makedirs(output_dir)
    
    vidcap = cv2.VideoCapture(video_info[0])
    count = -1
    while vidcap.isOpened():
        count+=1
        success, image = vidcap.read()
        if success: 
            if video_info[1] * 30 <= count and count <= video_info[2] * 30:
                cv2.imwrite(os.path.join(output_dir, editing_video+ ' %d.jpg') %(count), image)
        else:
            cv2.destroyAllWindows()
            vidcap.release()

- 이제 Json 파일도 지워봅시다.

In [41]:
len(json_start_end_info)

8998

In [26]:
pd_test2 = pd.DataFrame(json_start_end_info)
pd_test2.head()

,0,1,2
0,../data/CROWD_REAL005/CROWD_REAL015/키포인트Json/N...,1.5,6.4
1,../data/CROWD_REAL005/CROWD_REAL015/키포인트Json/N...,1.0,7.8
2,../data/CROWD_REAL005/CROWD_REAL015/키포인트Json/N...,1.5,5.4
3,../data/CROWD_REAL005/CROWD_REAL015/키포인트Json/N...,1.9,6.7
4,../data/CROWD_REAL005/CROWD_REAL015/키포인트Json/N...,1.1,5.4


- json file 삭제 시작~

In [71]:
for json_info in json_start_end_info:
    json_dir = json_info[0]
    json_start = json_info[1]
    json_end = json_info[2]
    for i, json_file_name in enumerate(sorted(os.listdir(json_dir))):
        if i < json_start * 30 or json_end * 30 < i: # 종료시간보다 전이거나 후면 삭제!
            os.remove(json_dir + '/' + json_file_name)

- keypoints preprocess

In [5]:
import json
import os
import glob
import numpy as np

In [6]:
# 경로 설정
kp_path = '../data/CROWD_REAL004/CROWD_REAL012/키포인트Json' # 기존 키포인트 Json
new_kp_dir = '../data/CROWD_REAL004/CROWD_REAL012/키포인트txt' # 새로 저장할 키포인트 txt
# os.listdir('../data/CROWD_REAL004/CROWD_REAL010/키포인트Json')
len(os.listdir(kp_path))

1000

In [7]:
json_files = glob.glob(os.path.join(kp_path,'**','*.json'), recursive=True)
json_files[:5]

['../data/CROWD_REAL004/CROWD_REAL012/키포인트Json/NIA_SL_FS0539_CROWD06_F.mp4/NIA_SL_FS0539_CROWD06_F__000000000085_keypoints.json',
 '../data/CROWD_REAL004/CROWD_REAL012/키포인트Json/NIA_SL_FS0539_CROWD06_F.mp4/NIA_SL_FS0539_CROWD06_F__000000000169_keypoints.json',
 '../data/CROWD_REAL004/CROWD_REAL012/키포인트Json/NIA_SL_FS0539_CROWD06_F.mp4/NIA_SL_FS0539_CROWD06_F__000000000052_keypoints.json',
 '../data/CROWD_REAL004/CROWD_REAL012/키포인트Json/NIA_SL_FS0539_CROWD06_F.mp4/NIA_SL_FS0539_CROWD06_F__000000000164_keypoints.json',
 '../data/CROWD_REAL004/CROWD_REAL012/키포인트Json/NIA_SL_FS0539_CROWD06_F.mp4/NIA_SL_FS0539_CROWD06_F__000000000107_keypoints.json']

In [8]:
keypointss = []
ZERO = False
for i, json_file in enumerate(json_files):
    with open(json_file, "r") as keypoints_json:
        data = json.load(keypoints_json)
 
    # pose에서 생략 가능한 하반신 30-35, 39-44, 57-74
    pose = data['people'][0]['pose_keypoints_2d']
    pose = pose[:30]+pose[36:39]+pose[45:57] # pose 75개 -> 45개
    face = data['people'][0]['face_keypoints_2d']
    hand_left = data['people'][0]['hand_left_keypoints_2d']
    hand_right = data['people'][0]['hand_right_keypoints_2d']
    
    # 전체 keypoints 합치고 confidence 값 제외
    entire_keypoints = np.array(pose+face+hand_left+hand_right, dtype=np.float32).reshape(-1,3)[:,:2].flatten() # 254
    keypointss.append(entire_keypoints) # 일단 keypointss 리스트에 다 저장 - 0값 채우기 위해서
    
    if i>1 and ZERO: # 앞에 0인 값이 있으면 i-2, i의 평균값으로 i-1의 0 채워주기
        zero_tf = keypointss[i-1]==0
        keypointss[i-1][zero_tf] = (keypointss[i-2][zero_tf]+keypointss[i][zero_tf])/2 # 앞,뒤 평균값으로 채워주기
        ZERO = False
        
    if 0 in entire_keypoints: # 0이 있으면 ZERO!
        ZERO = True

In [9]:
def normalization(array):
    scale_array = array.copy()
    scale_array[::2] /= 2048 # x좌표
    scale_array[1::2] /= 1152 # y좌표
    return scale_array

In [11]:
for json_file, keypoints in zip(json_files, keypointss):
    if 0 in keypoints:
        print(json_file, keypoints)

../data/CROWD_REAL004/CROWD_REAL012/키포인트Json/NIA_SL_FS0539_CROWD06_F.mp4/NIA_SL_FS0539_CROWD06_F__000000000085_keypoints.json [ 524.091  608.146  550.419  859.316  330.536  848.997    0.       0.
    0.       0.     765.282  864.769  828.193 1184.11   896.121 1482.76
  561.107 1592.83   409.166 1608.48   707.573 1587.56   477.251  561.044
  566.445  560.912  414.337  597.471  649.911  587.087  419.956  563.626
  424.465  594.057  427.846  622.234  434.608  649.283  443.625  676.333
  454.896  697.747  474.056  715.78   499.978  724.797  530.409  729.305
  559.713  723.67   582.254  712.399  605.922  695.493  623.956  670.698
  632.972  643.648  637.48   615.471  640.862  587.295  641.989  559.118
  441.371  535.45   452.641  525.306  469.547  524.179  483.072  527.56
  498.851  535.45   537.171  528.687  552.95   516.289  572.11   512.908
  595.779  521.925  607.049  535.45   520.265  560.245  519.138  579.405
  519.138  597.438  519.138  615.471  501.105  633.504  512.376  636.885
  5

In [46]:
os.makedirs(new_kp_dir)

In [47]:
def check_one_hot(file_name):
    txt = file_name.split('_')[3]
    tmp = []
    for c in txt:
        if c.isdigit():
            tmp.append(c)
    return int(''.join(tmp))

In [48]:
os.makedirs(new_kp_dir, exist_ok=True) # 폴더 만들고
new_kp_dir = new_kp_dir + '/'
for json_file, keypoints in zip(json_files, keypointss):
    txt_file_name = os.path.basename(os.path.dirname(json_file))
    f = open(new_kp_dir+txt_file_name[:-4]+'.txt', 'a')
    # 저장
    data = "%s, " %os.path.basename(json_file)[:-5] + ', '.join(list(map(str,normalization(keypoints)))) + ',' # keypoints 값에 normalization 적용해서
    one_hot = [0 for i in range(22)]
    person = check_one_hot(txt_file_name)
    one_hot[person-1] = 1
    data += str(one_hot) + '\n'

    f.write(data)
    f.close()